# Data Generation and Analysis workshop
## Wednesday Oct 9, 2024: Towards a Global Fit - part 1/2

This tutorial shows minimal methods to fit for an instrumental noise model and a stochastic GW background. It is oultined as follows:

0. Installation of required dependencies
1. Orbit file generation
2. Download GW TDI data from LDC and check them against PSD models
3. Download noise TDI data from LDC and check them against PSD models
4. Perform the estimation of noise parameters
5. Perform the estimation of GW parameter
6. Using more TDI channels (exercice)

## 0. Requirements
Install and import the required packages

In [ ]:
# LISA Constants
!pip install lisaconstants
# LISA Orbits
!pip install lisaorbits
# LISA Instrument
!pip install lisainstrument
# LISA GW Response
!pip install lisagwresponse
# pyTDI
!pip install pytdi

In [ ]:
# Install a tool to compute phase transition spectra
!pip install --upgrade git+https://dweir@bitbucket.org/dweir/ptplot.git
# Install a corner plot tool
!pip install corner
# Install backgrounds analysis tool
!pip install backgrounds

In [ ]:
# Install a sampler
!git clone https://github.com/willvousden/ptemcee.git
!pip install ./ptemcee

In [ ]:
import os
import logging
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp
import ptemcee
import corner
import lisaconstants
from h5py import File

from scipy.signal import welch
from scipy.interpolate import interp1d
import backgrounds

from ptplot.science.calculate_powerspectrum import PowerSpectrum
from lisaorbits import EqualArmlengthOrbits
from lisagwresponse import StochasticBackground, psd
from lisainstrument import Instrument
from lisainstrument.containers import ForEachMOSA
from pytdi import Data, michelson

logging.basicConfig(level=logging.INFO)

## 1. Generate an orbit file

In [ ]:
# Create directory for outputs
!mkdir data

In [ ]:
# Set directory for outputs
BASEDIR = './data'
# Set to true for quick tests
QUICKTEST = True

In [ ]:
# Set the orbit simulation duration
a_day = 24 * 3600 # s
a_year = 365.25 * a_day # s
duration = 2 * a_year
# Set orbit initial and sampling time
orbits_t0 = 0
orbits_dt = 100000
# We use a 2 year orbit (to allow for margin)
orbits_size = int(duration// orbits_dt) + 1
orbits_path = os.path.join(BASEDIR, 'orbits.h5')
# Create an orbit object
orbits = EqualArmlengthOrbits(
    t_init=orbits_t0
)

In [ ]:
# Remove file if it exists
if os.path.exists(orbits_path):
    os.remove(orbits_path)
# Generate and write orbit file
orbits.write(orbits_path, dt=orbits_dt, size=orbits_size, mode='w')

In [ ]:
# Read orbit file
with File(orbits_path, 'r') as hdf:

    t = orbits_t0 + np.arange(orbits_size) * orbits_dt

    sc_positions = hdf['tcb/x'][:]
    ltts = hdf['tcb/ltt'][:]
    pprs = hdf['tps/ppr'][:]

    plt.figure(figsize=(12, 9))
    plt.subplot(311)
    for i, coord in enumerate('xyz'):
        plt.plot(t, sc_positions[:, 0, i], label=rf'${coord}$')
    plt.ylabel('Spacecraft 1 [m]')
    plt.title('Spacecraft positions')
    plt.legend()
    plt.subplot(312)
    for i, coord in enumerate('xyz'):
        plt.plot(t, sc_positions[:, 1, i], label=rf'${coord}$')
    plt.ylabel('Spacecraft 2 [m]')
    plt.legend()
    plt.subplot(313)
    for i, coord in enumerate('xyz'):
        plt.plot(t, sc_positions[:, 2, i], label=rf'${coord}$')
    plt.ylabel('Spacecraft 3 [m]')
    plt.xlabel('Time [s]')
    plt.legend()

    plt.figure(figsize=(12, 4))
    plt.subplot(111)
    plt.plot(t, ltts)
    plt.ylabel('LTT [s]')
    plt.xlabel('Time [s]')
    plt.title('Light travel time')

## 2. Download and check signal data

### 2.1 Get data from LDC

We dowload a dataset containing the TDI measurements of a stochastic gravitational wave background (SGWB) from a first-order phase transition happening in the early universe.

In [ ]:
# We download a year-long dataset from the LDC website
# !wget "https://lisa-ldc.lal.in2p3.fr/media/uploads/cosmo/fopt-1b/tdi.h5"
!gdown 1VarTWIEtomHNWTQoikdnXU9XX93xDkBB
!mv tdi.h5 data/tdi_gw.h5

In [ ]:
# Load the LDC SGWB dataset
ldc_data_path = f'{BASEDIR}/tdi_gw.h5'
with File(ldc_data_path, 'r') as hdf:
    # Hubble constant
    H0 = 3.24E-18
    # The original dataset was missing a factor of sqrt{H0}
    x2_gw_ldc = hdf['X2'][:] * np.sqrt(H0)
    y2_gw_ldc = hdf['Y2'][:] * np.sqrt(H0)
    z2_gw_ldc = hdf['Z2'][:] * np.sqrt(H0)

In [ ]:
def asd(x, fs, **kwargs):
    """Compute ASD from time series.

    Args:
        x (array): time series
        fs (float): sampling frequency [Hz]
        kwargs: keyword arguments passed to scipy.signal.welch
    """
    if 'window' not in kwargs:
        kwargs['window'] = ('kaiser', 30)
    if 'nperseg' not in kwargs:
        kwargs['nperseg'] = 2**18
    if 'detrend' not in kwargs:
        kwargs['detrend'] = None
    freq, psd = welch(x, fs, **kwargs)
    return freq[5:], np.sqrt(psd[5:])

In [ ]:
# Choose the size of the Welch segments
nperseg = 2**17
# Choose the segment overlap
noverlap = nperseg // 2
# Sampling frequency
tdi_fs = 0.5 # Hz
# Compute periodogram
f, asd_x2 = asd(x2_gw_ldc[500:-500], tdi_fs, nperseg=nperseg,
                window="hann", noverlap=noverlap)

### 2.2 Define the stochastic background

We use `ptplot` [![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.6949107.svg)](https://doi.org/10.5281/zenodo.6949107) (<https://www.ptplot.org>,
 Weir 2023) to compute the energy density power spectrum of a stochastic background due to first-order phase transitions (see Caprini et al 2021).

We use values used in Fig. 6 of the Cosmology with LISA white paper (SNR of 10):

$$v_w = 0.9,$$
$$\alpha = 0.1,$$
$$\beta / H = 50,$$
$$T_\star = 200 \, \text{GeV},$$
$$g_\star = 100.$$

The tools gives $h^2 \Omega(f)$. We convert to a one-sided power density spectral density with

$$S_h(f) = h^2 \Omega(f) \frac{3 (H_0/h)^2}{4 \pi^2 f^3},$$

with $H_0/h = 100 \, \text{km} \text{s}^{-1} \text{Mpc}^{-1} \approx 3.24 \times 10^{-18} \text{s}^{-1}$.

We can compute the total energy in the sky and divide up into $N$ pixels, and 2 polarizations, to get the energy in each pixel,

$$S_\text{pixel}(f) = \frac{4 \pi}{2 N} S_h(f).$$

In [ ]:
# We offset the signal t0 to have orbits information at emission time
response_t0 = orbits_t0 + 100.0 # seconds
response_fs = tdi_fs
response_dt = 1 / response_fs
# Generate 1 year dataset
response_duration = a_day if QUICKTEST else a_year
response_size = int(response_duration // response_dt) + 1
response_nside = 4 if QUICKTEST else 8
response_npix = hp.nside2npix(response_nside)
response_path = os.path.join(BASEDIR, 'response.h5')
# Specifications of the stochastic background (energy density)
spectrum = PowerSpectrum(vw=0.9, alpha=0.1, BetaoverH=50, Tstar=200, gstar=100)
h2_Omega = lambda f: spectrum.power_spectrum(f)
# Conversion to one-sided power density spectral density
energy_density_psd = lambda f: h2_Omega(f) * (3 * H0 ** 2) / (4 * np.pi**2 * f**3)
# Conversion to one-sided power spectral density per pixel per polarization
pixel_psd = lambda f: energy_density_psd(f) * (4 * np.pi) / (2 * response_npix)

### 2.3 Compute the response model

The sky-averaged single-link response to gravitational waves can be computed numerically, although low-frequency approximations exist. See, for example, Babak et al., 2021 https://arxiv.org/abs/2108.01167

Here we use the analysis code delelopped in [Baghi et al. 2023](https://iopscience.iop.org/article/10.1088/1475-7516/2023/04/066).
The SGWB PSD can be written as
$$
S_{X2, \mathrm{gw}}(f) = S_{h}(f) R_{X2}(f)
$$
where $R$ is the sky-averaged response function for TDI X2:
$$
\begin{aligned}
R_{X2}\left(f, t_0\right) & = R_{X2, +}\left(f, t_0\right) + R_{X2, \times}\left(f, t_0\right) \\
R_{X2, p}\left(f, t_0\right) & = \int G_{X2, p}\left(f, t_0, \hat{\mathbf{k}}\right) G_{X2, p}^*\left(f, t_0, \hat{\mathbf{k}}\right) \mathrm{d}^2 \hat{\mathbf{k}}
\end{aligned}
$$
where $G_{X2}$ is a time and frequency dependent kernel encoding the antenna pattern.

In [ ]:
# Size of the time series
data_size = x2_gw_ldc.size

In [ ]:
# Instantiate the GW response analysis model
npixels = hp.nside2npix(8)
# Create a normalized map distributing the power accross the sky
skymap = np.ones(npixels) * np.sqrt(4 * np.pi / (2*npixels))
# Instantiate the SGWB object
sgwb_cls = backgrounds.StochasticBackgroundResponse(skymap, orbits=orbits_path)
# Create a coarse frequency grid where to compute the response
freq_grid = np.exp(np.linspace(np.log(tdi_fs/data_size), np.log(tdi_fs/2), 2000))
# Compute the response matrix for XYZ
g_mat_xyz = sgwb_cls.compute_tdi_kernel(freq_grid, response_t0, tdi_var='xyz',
                                        gen='2.0', parallel=True)

In [ ]:
# Compute the SGWB PSD
sh = energy_density_psd(f)
# Build the sky-averaged GW response function by interpolating
r_function = interp1d(freq_grid, g_mat_xyz[:, 0, 0].real, kind="cubic",
                      fill_value="extrapolate")
# Compute the TDI response to the SGWB
tdi_model_gw = np.sqrt(sh * r_function(f))

### 2.4 Compare with the data periodogram

In [ ]:
_, axes = plt.subplots(2, 1, figsize=(12, 7), sharex=True,
                       gridspec_kw={'height_ratios': [2, 1]})

axes[0].loglog(f, asd_x2, label='X2')
axes[0].loglog(f, tdi_model_gw, color='black', lw=2, label='model')
axes[0].set_ylim(1E-26, 1E-20)
axes[0].set_ylabel(r'TDI ASD [$\mathrm{Hz^{-1/2}}$]')
axes[0].legend()

axes[1].loglog(f, asd_x2 / tdi_model_gw, label='X2')
axes[1].set_ylim(1E-2, 1E1)
axes[1].set_xlabel('Frequency [Hz]')
axes[1].set_ylabel(r'Whitened noise')
axes[1].legend()

## 3. Download and check noise data

### 3.1 Get data from LDC

In [ ]:
# We can either use the dataset we just simulated, or download a year-long dataset from the LDC website
# !wget "https://lisa-ldc.lal.in2p3.fr/media/uploads/cosmo/noise-1a/tdi.h5"
!gdown 1DwRUzJ1JZZRmjXp1ZSL8JfvpvB5lKuKi
!mv tdi.h5 data/tdi_noise.h5

In [ ]:
# Similarly, instead of analysing the noise we generated, we download a year-long simulation.
ldc_noise_path = f'{BASEDIR}/tdi_noise.h5'

with File(ldc_noise_path, 'r') as hdf:
    x2_noise_ldc = hdf['X2'][:]
    y2_noise_ldc = hdf['Y2'][:]
    z2_noise_ldc = hdf['Z2'][:]

### 3.2 Define the noise model

#### OMS noise analytical model

In the inter-spacecraft interferometer (ISI, or science) carrier beatnotes, we should see the optical metrology system (OMS) noise. In terms of displacement, it's given by
$$
S_\text{oms}^\text{m}(f) = A^2 \Big[ 1 + \Big(\frac{f_\text{knee}}{f}\Big)^4 \Big].
$$
Multiplying by $(2\pi f / c)^2$ to express it as fractional frequency deviations, we can obtain the equivalent frequency shift by adding the $\nu_0^2$​ factor,
$$
S_\text{oms}^\text{Hz}(f) = \Big(\frac{2\pi f \nu_0}{c}\Big)^2 S_\text{oms}^\text{m}(f).
$$

In [ ]:
from lisaconstants import c

def oms_in_isi_carrier(freq, instru, mosa='12'):
    """Model for OMS noise PSD in ISI carrier beatnote fluctuations.

    Args:
        freq (float): frequencies [Hz]
        instru (Instrument): LISA instrument object
    """
    asd = instru.oms_isi_carrier_asds[mosa]
    fknee = instru.oms_fknees[mosa]
    psd_meters = asd**2 * (1 + (fknee / freq)**4)
    psd_hertz = (2 * np.pi * freq * instru.central_freq / c)**2 * psd_meters
    return np.sqrt(psd_hertz)

#### TM noise analytical models

In the test-mass interferometer (TMI) carrier beatnotes, we should see the test-mass noise. In terms of acceleration,
$$S_\delta^{\text{m}\,\text{s}^{-2}}(f) = A^2 [ 1 + \Big(\frac{f_\text{knee}}{f}\Big)^2 ].$$
Multiplying by $1/(2 \pi f )^2$ yields the noise as a velocity, and then dividing by c yields the noise as a Doppler shift. One needs to scale this by $2\nu_{0}$​ to get the equivalent frequency shift (the factor 2 is for the bouncing on the test mass),
$$S_\delta^\text{Hz}(f) = \Big(\frac{2 \nu_0}{2 \pi c f}\Big)^2 S_\delta^{\text{m}\,\text{s}^{-2}}(f).$$

In [ ]:
def testmass_in_tmi_carrier(freq, instru, mosa='12'):
    """Model for TM noise PSD in TMI carrier beatnote fluctuations.

    Args:
        freq (float): frequencies [Hz]
        instru (Instrument): LISA instrument object
    """
    asd = instru.testmass_asds[mosa]
    fknee = instru.testmass_fknees[mosa]
    psd_acc = asd**2 * (1 + (fknee / freq)**2)
    psd_hertz = (2 * instru.central_freq / (2 * np.pi * c * freq))**2 * psd_acc
    return np.sqrt(psd_hertz)

#### TDI instrumental noise model

The transfer functions through TDI are taken from [Dam Quang et al](https://arxiv.org/pdf/2211.02539.pdf). We use the common factor that depends on the average arm length L (in the code, directly given in seconds by the PPRs),
$$C_{XX}(f) = 16 \sin^2(2 \pi f L / c) \sin^2(4 \pi f L / c).$$

The transfer function for the ISI OMS noise PSD is given by
$$4 C_{XX}(f).$$

The transfer function for the test-mass noise PSD reads
$$C_{XX}(f) [3 + \cos(4 \pi f L / c)].$$

Note that the factor 2 has been removed since it's already included in the noise PSD.

Therefore, the total noise model can be written as
$$
S_{X2, n}(f) = S_{X2, \mathrm{oms}}(f) + S_{X2, \mathrm{tm}}(f)
$$
with
$$
\begin{align}
S_{X2, \mathrm{oms}}(f) & = 4 C_{XX}(f) S_{\mathrm{oms}}(f) \\
S_{X2, \mathrm{tm}}(f) & = C_{XX}(f) [3 + \cos(4 \pi f L / c)] S_{\mathrm{\delta}}(f)
\end{align}
$$

In [ ]:
def tdi_common(freq, instru, mosa='12'):
    """TDI common factor.

    Args:
        freq (float): frequencies [Hz]
        instru (Instrument): LISA instrument object

    Reference: https://journals.aps.org/prd/abstract/10.1103/PhysRevD.108.082004

    """
    armlength = np.mean(instru.pprs[mosa])
    return 16 * np.sin(2 * np.pi * freq * armlength)**2 \
        * np.sin(4 * np.pi * freq * armlength)**2


def tdi_tf_oms(freq, instru, mosa='12'):
    """TDI transfer function for ISI OMS noise.

    Args:
        freq (float): frequencies [Hz]
        instru (Instrument): LISA instrument object
    """
    psd = 4 * tdi_common(freq, instru, mosa)
    return np.sqrt(psd)


def tdi_tf_testmass(freq, instru, mosa='12'):
    """TDI transfer function for test mass noise.

    Args:
        freq (float): frequencies [Hz]
        instru (Instrument): LISA instrument object
    """
    armlength = np.mean(instru.pprs[mosa])
    psd = tdi_common(freq, instru, mosa) * (3 + np.cos(4 * np.pi * freq * armlength))
    return np.sqrt(psd)

### 3.3 Compute the noise model

In [ ]:
# We offset the instrument t0 to have orbits information at emission time
instru_t0 = orbits_t0 + 100.0
instru_fs = response_fs
instru_dt = 1 / instru_fs
# Generate 1 year dataset
instru_duration = response_duration
instru_size = int(instru_duration // instru_dt) + 1
# Instantiate the instrument class
instru = Instrument(
    t0=instru_t0,
    dt=instru_dt,
    size=instru_size,
    physics_upsampling=1,
    aafilter=None,
    lock='six',
    orbits=orbits_path,
    orbit_dataset='tcb/ltt',
)

instru.disable_dopplers()
instru.disable_all_noises()
# Noise curves from LDC Spritz
instru.testmass_asds = ForEachMOSA(2.4E-15)
instru.oms_isi_carrier_asds = ForEachMOSA(7.9e-12)

**EXERCICE 1**: Based on the functions `tdi_tf_testmass`, `testmass_in_tmi_carrier`, `tdi_tf_oms`, `oms_in_isi_carrier` defined below, write down the full ASD (or PSD) model for TDI $X_2$.

In [ ]:
### EXERCICE 1 ###

### 3.4 Compare with the data periodogram

In [ ]:
# Add noise and signal after converting everything in relative frequency deviations
x2 = x2_noise_ldc / instru.central_freq + x2_gw_ldc

In [ ]:
# Compute the periodogram of noise + signal data
f, asd_x2 = asd(x2[500:-500], tdi_fs, nperseg=nperseg, window="hann", noverlap=noverlap)

In [ ]:
_, axes = plt.subplots(1, 1, figsize=(12, 7))

axes.loglog(f, asd_x2, label='X2')
axes.loglog(f, tdi_model, color='black', lw=2, label='Total model')
axes.loglog(f, tdi_model_gw, color='tab:red', lw=2, label='GW model')
axes.loglog(f, model_noise/instru.central_freq, color='gray', lw=2, linestyle="dashed", label='Noise model')
axes.set_ylim(1E-25, 1E-18)
axes.set_xlim(1E-4, 1E-1)
axes.set_ylabel(r'TDI ASD [$\mathrm{Hz^{-1/2}}$]')
axes.set_xlabel("Frequency [Hz]")
axes.legend()


## 4. Data analysis: parameter estimation

### 4.1 Noise-only parameter estimation

The Welch periodogram is approximately proportional to a chi squared distribution with $\nu$ degrees of freedom:
$$
\bar{P}(f) \sim \frac{1}{\nu} S(f) \chi^2_{\nu}
$$
The effetive number of degrees of freedom depends on the type of tappering windows, as well as
the number of segments and their overlaps. See, for example, [Solomon 1991](https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.osti.gov/servlets/purl/5688766&ved=2ahUKEwjtw8rj3MmIAxXlgf0HHREsGyEQFnoECBgQAQ&usg=AOvVaw32eeZggUFFfstzQ2jIJnSF).

One can compute it as
$$
\nu = \frac{2 \cdot \mathrm{K}}{1+\sum_{\mathrm{k}=1}^{\mathrm{K}-1} \frac{\mathrm{K}-\mathrm{k}}{\mathrm{K}} \rho(\mathrm{k}, \mathrm{S})}
$$
where
$$
\rho(k, S)= \begin{cases}\left(\frac{1}{\mathrm{NPG}} \sum_{m=1}^{M-k S} w(m) w(m+k S)\right)^2, & 1 \leq k \leq \operatorname{int}(M / S) \\ 0, & S \geq M\end{cases}
$$
Here,
* $w$ is the window function
* $N$ is the size of the full time series
* $K$ is the number of segments
* $M$ is the number of points in each segment
* $S$ is the number of points to shift between segments, to allow for overlaps
* $\mathrm{NPG} = \sum_{n=1}^{M} w_{n}^2$ is the noise power gain

The likelihood is therefore given by (see [Hindmarsh et al 2024](https://arxiv.org/abs/2406.04894))
$$
\mathcal{L}(\bar{P} \mid S)=\prod_{n=0}^{M-1} \frac{1}{2^{(\nu / 2)} \Gamma(\nu / 2)}\left(\frac{\nu}{S\left(f_n\right)}\right)\left(\nu \frac{\bar{P}\left(f_n\right)}{S\left(f_n\right)}\right)^{(\nu / 2)-1} \exp \left(-\frac{\nu}{2} \frac{\bar{P}\left(f_n\right)}{S\left(f_n\right)}\right)
$$

To start with, we use a parametrized noise model with two parameters $A_{\mathrm{oms}}$ and $A_{\mathrm{tm}}$, determining the amplitude of the test-mass noise contribution and the optical metrology system noise contribution to the total noise, so that:
$$
S(f) = A_{\mathrm{oms}} S_{X2, \mathrm{oms}}(f) + A_{\mathrm{tm}} S_{X2, \mathrm{tm}}(f)
$$

#### Define the likelihood for the noise model

In [ ]:
class Likelihood:

    def __init__(self, freq, asd, nperseg, data_size, noverlap=None):
        """
        Parameters
        ----------
        freq : ndarray
            Frequency vector
        asd : ndarray
            ASD vector
        nperseg : int
            number of points in each segment
        data_size : int
            number of data points in the time series
        noverlap : int
            number of points to shift between segments, to allow for overlaps

        References
        ----------
        Hindmarsh et al., https://arxiv.org/abs/2406.04894


        """

        self.data_size = data_size
        self.x_per = np.abs(asd)**2
        self.freq = freq
        self.nperseg = nperseg

        # Effective number of degrees of freedom for the chi squared
        # distribution of the Welch periodogram with a 50% overlap using the
        # Hann window

        # Number of windows
        if noverlap==0:
            self.nw = int(self.data_size/self.nperseg)
            self.nu = 2 * self.nw
        else:
            # Only valid for the Hann window
            self.nw = 2 * self.data_size / self. nperseg - 1
            self.nu = (36/19) * self.nw**2 / (self.nw - 1)

        # OMS noise ASD in relative frequency deviations
        self.oms_psd = (tdi_tf_oms(self.freq, instru) * oms_in_isi_carrier(self.freq, instru) / instru.central_freq)**2
        # TM noise ASD in relative frequency deviations
        self.testmass_psd = (tdi_tf_testmass(self.freq, instru) * testmass_in_tmi_carrier(self.freq, instru) / instru.central_freq)**2

    def evaluate_noise_psd(self, params_n):
        """

        Parameters
        ----------
        params_n : ndarray
            Noise model parameters: amplitude of OMS noise + amplitude of TM

        """
        # Noise part
        psd = params_n[0] * self.oms_psd + params_n[1] * self.testmass_psd

        return psd

    def evaluate_loglike(self, params):
        """

        Parameters
        ----------
        params : ndarray
            Noise model parameters: amplitude of OMS noise + amplitude of TM
        """

        s_xx = self.evaluate_noise_psd(params)

        # loglike = -np.sum(self.x_per / s_xx + np.log(s_xx))
        loglike = np.sum(-(self.nu/2) * self.x_per / s_xx
                         +(self.nu/2 - 1) * np.log(self.x_per / s_xx)
                         - np.log(s_xx))
        return np.real(loglike)

In [ ]:
# We select the frequency bandwidth we want to analyze
f, asd_x2_noise = asd(x2_noise_ldc[500:-500] / instru.central_freq, tdi_fs,
                      nperseg=nperseg,
                      window="hann", noverlap=noverlap)
iband = np.where((f>=1e-4) & (f<=1e-2))
f_band = f[iband]
asd_x2_noise_band = asd_x2_noise[iband]

In [ ]:
_, axes = plt.subplots(1, 1, figsize=(12, 7))

axes.loglog(f_band, asd_x2_noise_band, label='X2')
axes.loglog(f_band, model_noise[iband]/instru.central_freq,
            color='black', lw=2, label='total noise model')
axes.set_ylabel(r'TDI ASD [Hz/$\sqrt{\mathrm{Hz}}$]')
axes.set_xlabel("Frequency [Hz]")
axes.legend()

In [ ]:
# Instantiate the likelihood class
loglike = Likelihood(f_band, asd_x2_noise_band, nperseg, len(x2[500:-500]), noverlap=noverlap)

In [ ]:
# Test of the likelihood
params = np.array([1, 1])
loglike.evaluate_loglike(params)
# List of deviations from true value for OMS noise amplitude
param1_list = np.linspace(0.5, 2, 50)
loglike_list = [loglike.evaluate_loglike(np.asarray([p, 1.0]))
for p in param1_list]

# Plot the loglikelihood values vs OMS noise parameter deviations
plt.figure(0)
plt.plot(param1_list - 1.0, loglike_list)
plt.xlabel('$\Delta$OMS noise')
plt.ylabel('Log-likelihood')
plt.show()

In [ ]:
# List of deviations from true value for TM noise amplitude
param2_list = np.linspace(0.5, 2, 50)
loglike_list = [loglike.evaluate_loglike(np.asarray([1.0, p]))
for p in param2_list]

# Plot the loglikelihood values vs OMS noise parameter deviations
plt.figure(0)
plt.plot(param2_list - 1.0, loglike_list)
plt.xlabel('$\Delta$TM noise')
plt.ylabel('Log-likelihood')
plt.show()

#### Define the prior

In [ ]:
class UniformPrior:

    def __init__(self, a, b):

        self.a = a
        self.b = b
        if isinstance(a, np.ndarray):
            self.ndim = self.a.size
        elif isinstance(a, list):
            self.ndim = len(a)
        else:
            self.ndim = 1

    def evaluate(self, x):

        # First check that knots are sorted
        if (np.all(self.a < x)) & (np.all(self.b > x)):
            # Check that values are within bounds and knots are sorted
            return 0.0  # knot_prior_cls.evaluate(knots)
        else:
            return -np.inf

    def initialize_single_param(self):
        """Initialize one parameter state

        Parameters
        ----------

        Returns
        -------
        p_0 : ndarray
            parameter vector, size ndim
        """

        p_0 = np.random.uniform(low=self.a, high=self.b)

        return np.atleast_1d(p_0)

In [ ]:
a = np.asarray([0.5, 0.5])
b = np.asarray([2, 2])

logprior = UniformPrior(a, b)

In [ ]:
# Set up the sampler
nwalkers = 8
ntemps = 5
ndim = 2
niter = 2000
# os.environ["OMP_NUM_THREADS"] = "1"
from ptemcee.interruptible_pool import Pool
mapper = Pool().map
# mapper = map

# Intialize the parameter state
pos = np.random.uniform(low=a, high=b, size=(ntemps, nwalkers, ndim))
# Instantiate the sampler
sampler = ptemcee.Sampler(nwalkers, ndim, loglike.evaluate_loglike,
                          logprior.evaluate,
                          betas=ptemcee.make_ladder(ndim, ntemps=ntemps, Tmax=None),
                          mapper=mapper)
# Run the sampler
# ensemble = sampler.sample(pos, rstate=np.random.get_state())
chain = sampler.chain(pos)
for counter, x in enumerate(chain.iterate(niter)):
    if counter % 100 == 0:
        print(counter)

In [ ]:
print(chain.x.shape)
plt.figure(figsize=(12, 12))
plt.subplot(311)
plt.plot(np.arange(niter), chain.x[:, 0, 0, 0])
plt.ylabel('OMS noise amplitude')

plt.subplot(312)
plt.plot(np.arange(niter), chain.x[:, 0, 0, 1])
plt.ylabel('TMI noise amplitude')
plt.xlabel('Step')

plt.show()

In [ ]:
# Reshape samples
burnin = 1000
nwalkers = chain.x.shape[2]
i_max = chain.x.shape[0]
samples_plot = chain.x[burnin:i_max, 0, :, :].reshape((nwalkers*(i_max-burnin),
                                                       chain.x.shape[-1]))
print(samples_plot.shape)
# Plot intrinsic parameters
figure = corner.corner(samples_plot[:, 0:4],
                       labels=[r"$A_{\mathrm{OMS}}$", r"$A_{\mathrm{TM}}$"],
                       show_titles=True,
                       truths=np.ones(2),
                       title_kwargs={"fontsize": 12})

### 4.2 Signal + noise parameter estimation

Now we analyse the data stream assuming the presence of a stochastic background of cosmological origin. The PSD model of channel $X_2$ can now be written as
$$
S(f) = A_{\mathrm{oms}} S_{X2, \mathrm{oms}}(f) + A_{\mathrm{tm}} S_{X2, \mathrm{tm}}(f) + S_{X2, \mathrm{gw}}(f, \alpha_{\mathrm{pt}})
$$



In [ ]:
# Now we consider a time series with a SGWB + noise
asd_x2_band = asd_x2[iband]

In [ ]:
_, axes = plt.subplots(1, 1, figsize=(12, 7))

axes.loglog(f_band, asd_x2_band, label='X2')
axes.loglog(f_band, model_noise[iband]/instru.central_freq,
            color='black', lw=2, label='noise model')
axes.loglog(f_band, tdi_model_gw[iband],
            color='red', lw=2, label='GW model')
axes.set_ylabel(r'TDI ASD [1/$\sqrt{\mathrm{Hz}}$]')
axes.legend()

**EXERCICE 2**: Modify the likelihood class below to include the GW signal into the model, governed by the parameter $\alpha_{\mathrm{pt}}$.

*Help*: to update the GW background PSD model with a new value of $\alpha_{\mathrm{pt}}$, one can write:


```
spectrum.alpha = alpha_pt
```


In [ ]:
### EXERCICE 2 ###

class LikelihoodFull:

    def __init__(self, freq, asd, nperseg, data_size, noverlap=None):
        """
        Parameters
        ----------
        freq : ndarray
            Frequency vector
        asd : ndarray
            ASD vector
        nperseg : int
            number of points in each segment
        data_size : int
            number of data points in the time series
        noverlap : int
            number of points to shift between segments, to allow for overlaps

        References
        ----------
        Hindmarsh et al., https://arxiv.org/abs/2406.04894


        """

        self.data_size = data_size
        self.x_per = np.abs(asd)**2
        self.freq = freq
        self.nperseg = nperseg

        # Effective number of degrees of freedom for the chi squared
        # distribution of the Welch periodogram with a 50% overlap using the
        # Hann window

        # Number of windows
        if noverlap==0:
            self.nw = int(self.data_size/self.nperseg)
            self.nu = 2 * self.nw
        else:
            # Only valid for the Hann window
            self.nw = 2 * self.data_size / self. nperseg - 1
            self.nu = (36/19) * self.nw**2 / (self.nw - 1)

        # OMS noise ASD in relative frequency deviations
        self.oms_psd = (tdi_tf_oms(self.freq, instru) * oms_in_isi_carrier(self.freq, instru) / instru.central_freq)**2
        # TM noise ASD in relative frequency deviations
        self.testmass_psd = (tdi_tf_testmass(self.freq, instru) * testmass_in_tmi_carrier(self.freq, instru) / instru.central_freq)**2

        # Complete here

In [ ]:
# Instantiate the likelihood class
loglike = LikelihoodFull(f_band, asd_x2_band, nperseg, data_size,
                         noverlap=noverlap)
# Test of the likelihood
params = np.array([1, 1, 0])
loglike.evaluate_loglike(params)

In [ ]:
# List of deviations from true value for OMS noise amplitude
param_gw_list = np.linspace(-1, 1, 50)
loglike_list = [loglike.evaluate_loglike(np.asarray([1.0, 1.0, p]))
for p in param_gw_list]

# Plot the loglikelihood values vs GW parameter deviations
plt.figure(0)
plt.plot(param_gw_list, loglike_list)
plt.xlabel('$\Delta$SGWB amplitude')
plt.ylabel('Log-likelihood')
plt.show()

In [ ]:
# Prior bounds for noise and signal parameters
a = np.asarray([0.5, 0.5, 0.0])
b = np.asarray([2, 2, 1.0])
logprior = UniformPrior(a, b)

In [ ]:
# Set up the sampler
nwalkers = 12
ntemps = 5
ndim = 3
niter = 2000
# Intialize the parameter state
pos = np.random.uniform(low=a, high=b, size=(ntemps, nwalkers, ndim))
# Instantiate the sampler
sampler = ptemcee.Sampler(nwalkers, ndim, loglike.evaluate_loglike,
                          logprior.evaluate,
                          betas=ptemcee.make_ladder(ndim, ntemps=ntemps,
                                                    Tmax=None),
                          mapper=mapper)
# Run the sampler
# ensemble = sampler.sample(pos, rstate=np.random.get_state())
chain = sampler.chain(pos)
for counter, x in enumerate(chain.iterate(niter)):
    if counter % 100 == 0:
        print(counter)

In [ ]:
print(chain.x.shape)
plt.figure(figsize=(12, 12))
plt.subplot(311)
plt.plot(np.arange(niter), chain.x[:, 0, 0, 0])
plt.ylabel('OMS noise amplitude')

plt.subplot(312)
plt.plot(np.arange(niter), chain.x[:, 0, 0, 1])
plt.ylabel('TM noise amplitude')
plt.xlabel('Step')

plt.subplot(313)
plt.plot(np.arange(niter), chain.x[:, 0, 0, 2])
plt.ylabel('SGWB amplitude')
plt.xlabel('Step')


plt.show()

In [ ]:
# Reshape samples
burnin = 1000
nwalkers = chain.x.shape[2]
i_max = chain.x.shape[0]
samples_plot = chain.x[burnin:i_max, 0, :, :].reshape((nwalkers*(i_max-burnin),
                                                       chain.x.shape[-1]))
print(samples_plot.shape)
# Plot intrinsic parameters
figure = corner.corner(samples_plot[:, 0:4],
                       labels=[r"$A_{\mathrm{oms}}$", r"$A_{\mathrm{tm}}$",
                               r"$\alpha_{\mathrm{pt}}$"],
                       show_titles=True,
                       truths=np.asarray([1.0, 1.0, 0.1]),
                       title_kwargs={"fontsize": 12})

## 6. Using more TDI channels

**EXERCICE 3** Try writing a noise-only likelihood class that now analyse TDI channels A and E. We give some hints below.

In [ ]:
### EXERCICE 3 ###

# We go from XYZ to AET by performing the following operation:
def xyz2aet(x, y, z):
    a,e,t = ((z - x)/np.sqrt(2.0),
            (x - 2.0*y + z)/np.sqrt(6.0),
            (x + y + z)/np.sqrt(3.0))
    return a,e,t


# The TDI PSD models for A, E, T are
def tdi_tf_oms(freq, instru, mosa='12', tdi='AET'):
    """TDI transfer function for ISI OMS noise.

    Args:
        freq (float): frequencies [Hz]
        instru (Instrument): LISA instrument object
    """
    if tdi=='XYZ':
      psd = 4 * tdi_common(freq, instru)
      return np.sqrt(psd)
    elif tdi=='AET':
      armlength = np.mean(instru.pprs[mosa])
      psd_AE = 2 * tdi_common(freq, instru) * ( 2 + np.cos(2 * np.pi * freq * armlength))
      psd_T = 4*tdi_common(freq, instru) * (1 - np.cos(2 * np.pi * freq * armlength))
      return np.sqrt(psd_AE), np.sqrt(psd_T)

def tdi_tf_testmass(freq, instru, mosa='12', tdi='AET'):
    """TDI transfer function for test mass noise.

    Args:
        freq (float): frequencies [Hz]
        instru (Instrument): LISA instrument object
    """
    armlength = np.mean(instru.pprs[mosa])
    if tdi=='XYZ':
      psd = tdi_common(freq, instru) * (3 + np.cos(4 * np.pi * freq * armlength))
      return np.sqrt(psd)
    elif tdi=='AET':
      psd_AE = tdi_common(freq, instru) * (3 + 2*np.cos(2 * np.pi * freq * armlength) + np.cos(4 * np.pi * freq * armlength))
      psd_T = 32*tdi_common(freq, instru) * np.sin(np.pi * freq * armlength)**4
      return np.sqrt(psd_AE), np.sqrt(psd_T)